## Notebook to solve 1D elliptic, self adjoint eigenvalue problem

Here we solve a (formally) self adjoint, regular elliptic problem in 1D, the problem is of the form:

$$ -\frac{d}{dx}(p(x)\frac{d}{dx}u) + q(x) u(x) = \lambda r(x) u(x)$$

with $p(x) > 0$. We solve it in an interval $[0,L]$ and use boundary conditions $\{u_0=0, u_1=0\}$ at the left-right boundaries. 



In [ ]:
using Gridap
using GridapMakie, GLMakie
GLMakie.activate!(inline=true)
using FileIO
#using Plots
#mkdir("models")
#mkdir("images")
using GridapGmsh
using LinearAlgebra
using SparseArrays
using SuiteSparse
#import Pkg; Pkg.add("Arpack")
using Arpack


We define several instances:

#### Test 1: is just the second derivatives (p=1, q=0, r=1)
The solutions are $u_n = sin(n*x)$ in the interval $[0,\pi]$, and the eigenvalues are $\lambda_n = n^2$.

#### Test 2: has (p=x, q=0, r=1/x)
The solutions are: $(\lambda = (\frac{n\pi}{\ln L})^2\;\;\;\; n>0, u_n = \sin(\frac{n\pi}{\ln L}\ln x)$, in the interval $[1,L]$.

#### Test 3: the harmonic oscillator (p=1, q=x^2, r=1)
The solutions are: $(\lambda = 1 + 2n, \;\;\; n\geq 0, u_n = e^{-x^2/2}H_n)$, where $H_n$ are the Hermite polynomials.


In [ ]:
test_1 = false 
test_2 = false
test_3 = false
test_1 = true
#test_2 = true 
#test_3 = true 


In [ ]:
#you can create the grids or load from files.
#include("models/mesh_generator.jl")


Define an equi-spaced grid with N nodes of length L

In [ ]:
# Create mesh with N point

if test_1 
    
    grid_type = "1D"
    h = 1/200
    par = (0,π, h) #for test (Length_x, Length_y, h) 
    
elseif test_2
    
    grid_type = "1D"
    h = 1/200
    L = 2
    par = (1,L, h) #for test (Length_x, Length_y, h) 
    
elseif test_3
    
    grid_type = "1D"
    h = 1/20000
    par = (-10,10, h) #for test (Length_x, Length_y, h) 
    
else
    
    grid_type = "1D"
    h = 1/1000
    par = (0.,20., h) #for test (Length_x, Length_y, h) 
    
end

#model = make_model(grid_type, par)
#model = GmshDiscreteModel("models/1D.msh")

@show l = 1 ÷ par[3] #number of elements in the mesh
@show domain = (par[1], par[2])
model = CartesianDiscreteModel(domain, l)
#= for the mesh in models use:
    boundary_tags = ["left", "right"]
    dirichlet_tags= ["left", "right"]
    dirichlet_values = [0.0, 0.0] # 0.0+im*0.0
=#
# For the Cartesian mesh use:    
    dirichlet_tags= ["boundary"]
    dirichlet_values = [0.0] # 0.0+im*0.0


In [ ]:
Ω = Triangulation(model)
degree = 3
dΩ = Measure(Ω,degree)


In [ ]:
fig, ax = plot(Ω)
scatter!(Ω, marker=:star8, markersize=4, color=:blue)
#wireframe!(Ω, color=:black, linewidth=2)
fig


In [ ]:
#Γ = BoundaryTriangulation(model,tags=boundary_tags)
#dΓ = Measure(Γ,degree)
#fig, ax = plot(Γ, linewidth=8)
#ax.aspect = AxisAspect(1)
#wireframe!(Γ, color=:black, linewidth=1)
#fig

In [ ]:
order = 2
reffe = ReferenceFE(lagrangian,Float64,order)

V = TestFESpace(model,reffe;vector_type=Vector{ComplexF64},conformity=:H1,dirichlet_tags = dirichlet_tags)
U = TrialFESpace(V,dirichlet_values)


Since we shall be solving it with finite elements we first define the fuction space for these elements:

We define the two free fuctions, $g$ and $pot$. To test the code we have three test cases, for which the functions are simple and we know the solution explicitly. The fourth case is just some potential???

In [ ]:
# WARNING: we have changed the sign of p with respect to the previous definition. 
if test_1
    p(x) = 1
    q(x) = 0
    r(x) = 1
elseif test_2
    p(x) = x[1]
    q(x) = 0
    r(x) = 1/x[1]
elseif test_3
    p(x) = 1
    q(x) = x[1]*x[1]
    r(x) = 1
else
    p(x) = (1 + x[1] + 0.5*x[1]*x[1])/(10 + 0.5*x[1]*x[1]) # don't remember this potential...
    q(x) = -10*x[1]/(1 + 5*x[1]*x[1])
    r(x) = 1
end


In [ ]:
x = range(par[1], par[2], length=1000)
yq = q.(x)
yp = p.(x)
yr = r.(x)
lines(x, yq, label= "q")
lines!(x, yp, label= "p")
lines!(x, yr, label= "r")
axislegend()
current_figure()


Now we define the variational problem

In [ ]:
a(u,v) = ∫(p*∇(v)⋅∇(u) + q*v*u)*dΩ
m(u,v) = ∫(r*u*v)dΩ


Here we introduce the solver and its parámeters, and the solve the system.

In [ ]:
include("eigen.jl")


In [ ]:
nev = 12 # number of eigenvalues asked to evaluate.
prob = EigenProblem(a, m, U, V; nev=nev, tol=10^(-6), maxiter=100, explicittransform=:none, sigma=-1.0)
#prob = EigenProblem(a, m, U, V; nev=nev, which=:LM, explicittransform=:auto, tol=10^(-6), maxiter=100, sigma=0)
ξ, uₕs = solve(prob);


In [ ]:
ξ.^(1/2)

We now illustrate how to manipulate the solutions (eigenfunctions and eigenvalues)

In [ ]:
uₕ = uₕs[10]
x = range(par[1], par[2], length=1000)
lines(x, evaluate(real(uₕ),Gridap.Point.(x)), label= "u")


In [ ]:
fig = Figure()

ax = Axis(fig[1, 1], xlabel = "eigenvalue number", ylabel = "value",
    title = "Eigenvalues", yscale = log10, xscale = log10)

scatter!(sort(real(ξ[1:nev])), label = "computed")

if test_1
    scatter!(ax,[n^2 for n ∈ 1:nev], marker=:star8, label = "true")
elseif test_2
    scatter!([(n*π/log(L))^2 for n ∈ 1:nev], marker=:star8, label = "true")
elseif test_3
    scatter!([1 + 2*(n-1) for n ∈ 1:nev], marker=:star8, label = "true")
end
axislegend()
current_figure()
fig

In [ ]:
ξ

And here we can save them into a file to see it with visit, for instance. 

In [ ]:
n=4 # for test_3 use n=4

uₕ = uₕs[n]
# normalize the eigenvector
u2 = sum(∫(uₕ*uₕ)*dΩ)
uₕ = uₕ/sqrt(u2)

if test_1
    u_n(x) = -sin(n*x)/sqrt(π/2)
elseif test_2
    freq=n*π/log(L)
    u_n(x) = sqrt((4*freq^2+1)/(2(L-1)*freq^2))*sin(freq*log(x))
elseif test_3
    u_n(x) = -exp(-x^2/2)*(8x^3-12x)/sqrt(85.0778) 
end

x = range(par[1], par[2], length=1000)
scatter(x, evaluate(real(uₕ),Gridap.Point.(x)), label= "computed")
lines!(x, u_n.(x), label= "exact")
scatter!(x, 100000*(evaluate(real(uₕ),Gridap.Point.(x))-u_n.(x)), label= "error x 10^5")
axislegend()
current_figure()        